<a href="https://colab.research.google.com/github/nureeee/DeepLearning/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%EC%BD%94%EB%A1%9C%EB%82%98_%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

서울시 코로나 동향

In [11]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [1]:
url = 'https://www.seoul.go.kr/coronaV/coronaStatus.do'

In [5]:
# 데이터 프레임화 할 수 있는 table 태그를 알아서 수집
table = pd.read_html(url)

In [6]:
len(table)

6

In [9]:
table[3]

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황


In [15]:
url = 'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1'
url = f"{url}$start=0&length=100" # 홈페이지 네트워크에 파라미터 확인하기 
url
#get 방식

'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1$start=0&length=100'

# 존체 페이지를 구하려면...?

In [16]:
response = requests.get(url)
data_json = response.json() # 또 다른 방식은 강사님 ...참고
data_json

{'data': [["<p class='corona19_no'>55415</p>",
   '169967',
   '2021-07-12',
   '영등포구',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>55414</p>",
   '169293',
   '2021-07-12',
   '타시도',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>55413</p>",
   '170295',
   '2021-07-12',
   '성동구',
   '-',
   '기타 확진자 접촉',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>55412</p>",
   '170281',
   '2021-07-12',
   '광진구',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>55411</p>",
   '170012',
   '2021-07-12',
   '타시도',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>55410</p>",
   '169649',
   '2021-07-12',
   '타시도',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>55409</p>",
   '170122',
   '2021-07-12',
   '성동구',
   '-',
   '기타 확진자 접촉',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>55408</p>",
   '169985',
   '2021-07-12',
   '강남구',
   '-',
   '감염경로 조사중'

In [17]:
records_total = data_json['recordsTotal']
records_total

25415

In [18]:
end_page = round(records_total / 100) + 1 if records_total % 100 != 0 else round(records_total / 100) # (전체 데이터 / 100 개씩) + 1 ( 짜투리 데이터들도 하나의 페이지에 더 보여줘야 하니까 )
end_page

255

In [20]:
data = data_json['data']
pd.DataFrame(data)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>55415</p>,169967,2021-07-12,영등포구,-,감염경로 조사중,<b class=''>-</b>
1,<p class='corona19_no'>55414</p>,169293,2021-07-12,타시도,-,감염경로 조사중,<b class=''>-</b>
2,<p class='corona19_no'>55413</p>,170295,2021-07-12,성동구,-,기타 확진자 접촉,<b class=''>-</b>
3,<p class='corona19_no'>55412</p>,170281,2021-07-12,광진구,-,감염경로 조사중,<b class=''>-</b>
4,<p class='corona19_no'>55411</p>,170012,2021-07-12,타시도,-,감염경로 조사중,<b class=''>-</b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>55320</p>,169893,2021-07-12,용산구,-,감염경로 조사중,<b class=''>-</b>
96,<p class='corona19_no'>55319</p>,169868,2021-07-12,노원구,-,강북구 소재 직장 관련('21.7.),<b class=''>-</b>
97,<p class='corona19_no'>55318</p>,169867,2021-07-12,노원구,-,타시도 확진자 접촉,<b class=''>-</b>
98,<p class='corona19_no'>55317</p>,170164,2021-07-12,금천구,-,타시도 확진자 접촉,<b class=''>-</b>


# 함수화

In [23]:
# def get_seoul_covid(page_no):
#     start = ( page_no - 1) * 100
#     url = f'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw={page_no}&start={start}&length=100'
#     res = requests.get(url)
#     json_data = res.json()

#     return json_data


In [ ]:
def get_seoul_covid(page_no):
  '''
    page_no : draw값
  '''
  start = ( page_no - 1 ) * 100
  url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw={page_no}&start={start}&length=100"
  res = requests.get(url)
  json_data = res.json() # json 데이터 불러오기

  return json_data


In [ ]:
get_seoul_covid(-1)


# 페이지 크롤링 해보기

In [28]:
from tqdm import tqdm_notebook

page_list = []

example_end_page = 3

for page_no in tqdm_notebook(range(1, example_end_page+1)):
    data = get_seoul_covid(page_no)
    covid_df = pd.DataFrame(data['data'])
    page_list.append(covid_df)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [29]:
pd.concat(page_list)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>55415</p>,169967,2021-07-12,영등포구,-,감염경로 조사중,<b class=''>-</b>
1,<p class='corona19_no'>55414</p>,169293,2021-07-12,타시도,-,감염경로 조사중,<b class=''>-</b>
2,<p class='corona19_no'>55413</p>,170295,2021-07-12,성동구,-,기타 확진자 접촉,<b class=''>-</b>
3,<p class='corona19_no'>55412</p>,170281,2021-07-12,광진구,-,감염경로 조사중,<b class=''>-</b>
4,<p class='corona19_no'>55411</p>,170012,2021-07-12,타시도,-,감염경로 조사중,<b class=''>-</b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>55120</p>,169928,2021-07-12,성북구,-,기타 확진자 접촉,<b class=''>-</b>
96,<p class='corona19_no'>55119</p>,169711,2021-07-12,도봉구,-,병원 및 요양시설,<b class=''>-</b>
97,<p class='corona19_no'>55118</p>,169698,2021-07-12,구로구,-,기타 확진자 접촉,<b class=''>-</b>
98,<p class='corona19_no'>55117</p>,169629,2021-07-12,동대문구,-,동대문구 소재 고시텔 관련('21.6.),<b class=''>-</b>


In [ ]:
page_list = []

for page_no in tqdm_notebook(range(1, end_page+1)):
  data = get_seoul_covid(page_no)
  covid_df = pd.DataFrame(data['data'])
  page_list.append(covid_df)

covid_final_df = pd.concat(page_list)
covid_final_df.info()

In [32]:
from tqdm import tqdm_notebook

page_list = []
for page_no in tqdm_notebook(range(1, end_page+1)):
    data = get_seoul_covid(page_no)
    covid_df = pd.DataFrame(data['data'])
    page_list.append(covid_df)

covid_final_df = pd.concat(page_list)
covid_final_df.info()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.



<class 'pandas.core.frame.DataFrame'>
Int64Index: 25415 entries, 0 to 14
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       25415 non-null  object
 1   1       25415 non-null  object
 2   2       25415 non-null  object
 3   3       25415 non-null  object
 4   4       25415 non-null  object
 5   5       25415 non-null  object
 6   6       25415 non-null  object
dtypes: object(7)
memory usage: 1.6+ MB


In [33]:
covid_final_df.columns=['연번', '환자', '확진일', '거주지', '여행력', '접촉력', '퇴원현황']
covid_final_df.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,<p class='corona19_no'>55415</p>,169967,2021-07-12,영등포구,-,감염경로 조사중,<b class=''>-</b>
1,<p class='corona19_no'>55414</p>,169293,2021-07-12,타시도,-,감염경로 조사중,<b class=''>-</b>
2,<p class='corona19_no'>55413</p>,170295,2021-07-12,성동구,-,기타 확진자 접촉,<b class=''>-</b>
3,<p class='corona19_no'>55412</p>,170281,2021-07-12,광진구,-,감염경로 조사중,<b class=''>-</b>
4,<p class='corona19_no'>55411</p>,170012,2021-07-12,타시도,-,감염경로 조사중,<b class=''>-</b>


In [34]:
# csv 파일로 저장
covid_final_df.to_csv("seoul_covid.csv", encoding='utf-8', index=False)